# Preply Web Scrape

### Exploring the Web Site HTML

In [3]:
print("Start raw scrape")

Start raw scrape


In [4]:
############ DAILY REPORTS AUTOMATION ############
from selenium.common.exceptions import ElementClickInterceptedException
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.by import By
from time import sleep

In [7]:
import os
# Chrome Driver Parameters Setup
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
chrome_driver_path="C:\Program Files\Google\Chrome\Application\chromedriver.exe"
# Initialize Chrome Driver
driver = webdriver.Chrome(options=options)
driver.get("https://preply.com/en/online/english-tutors")

In [ ]:
profiles=driver.find_element(By.CSS_SELECTOR, "#__next > div.styles_PageWrap__lIYEc > main > div.styles_ResultsWrapper__kAXts > div.styles_MainContentWrapper__h3r02 > ul > li:nth-child(3)")
profiles.screenshot("output.png")
img=driver.find_element(By.CSS_SELECTOR,  "#__next > div.styles_PageWrap__lIYEc > main > div.styles_ResultsWrapper__kAXts > div.styles_MainContentWrapper__h3r02 > ul > li:nth-child(3) > section > div > div.styles_SearchCardAvatarWrapper__u_zNw > div > a > picture > img")

In [38]:
### Selecting Currency 
from selenium.webdriver.support.ui import Select
curr_utton = "#__next > section > div._15uGWh._3rnoor.X4eHUe._1o4EVj.hUFwKI._1-5zEl._2x1aeM._1WH1_Q._2JlASK.GWHMDM._1kf68Q._3nnn2i > div.styles_HeaderDropdownWrapper__uIOg5 > button"


def refresh_currency():
    driver.find_element(By.CSS_SELECTOR,curr_utton).click()
    select = Select(driver.find_element(By.NAME, 'currency'))
    sleep(1)
    select.select_by_visible_text("USD")
    driver.find_element(By.CSS_SELECTOR,curr_utton).click()
    sleep(1)
    driver.find_element(By.CSS_SELECTOR,curr_utton).click()
    sleep(2)
    select = Select(driver.find_element(By.NAME, 'currency'))
    select.select_by_visible_text("EUR")
    
try:
    refresh_currency()
except:
    print("Could not refresh currency, try again.")

In [43]:
profile_template = "#__next > div.styles_PageWrap__lIYEc > main > div.styles_ResultsWrapper__kAXts > div.styles_MainContentWrapper__h3r02 > ul > li:nth-child({0})"
{"Name":"","teacher_of":""}
profile = driver.find_element(By.CSS_SELECTOR,"#__next > div.styles_PageWrap__lIYEc > main > div.styles_ResultsWrapper__kAXts > div.styles_MainContentWrapper__h3r02 > ul > li:nth-child(3)")


def create_json_profile(text):
    
    return text.split("\n")
create_json_profile(profile.text)


['Super tutor',
 'Gordon G.',
 'English',
 '5',
 '18',
 'reviews',
 '360',
 'MXN',
 'per hour',
 '18 active students',
 '•',
 '1,082 lessons',
 'Speaks:',
 'EnglishNative PortugueseUpper-Intermediate+1',
 'Portuguese/Spanish speaker, focus on English speaking *** NEW TIMETABLE NOVEMBER 9TH 2022 - ACCEPTING NEW STUDENTS!! ***',
 "Hello, my name is Gordon and I'm from Ireland. I have 43 years. No wait, I don't have 43 years! I am 43 years old.",
 'I have taught over 3000 hours of English classes to people that speak Portuguese since 2014.',
 'Read more',
 'Book trial lesson',
 'Message']

In [29]:
profile_template = "#__next > div.styles_PageWrap__lIYEc > main > div.styles_ResultsWrapper__kAXts > div.styles_MainContentWrapper__h3r02 > ul > li:nth-child({0})"


n_of_pages=driver.find_element(By.CSS_SELECTOR,  "#__next > div.styles_PageWrap__lIYEc > main > div.styles_ResultsWrapper__kAXts > div.styles_MainContentWrapper__h3r02 > div > div > ul > li:nth-child(5) > a").text

for index_page in range(0,n_of_pages):
    driver.find_element(By.CSS_SELECTOR,profile_template)


'1217'